# Classification

While regression is used to predict a numeric value, classification is used to seperate data points into classes of different labels.

## Imports and Setup
 Here we will be using the Iris Dataset

In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
import pandas as pd

---

## Dataset

This specific dataset seperates flowers into 3 different classes of species.
- Setosa
- Versicolor
- Virginica


The information provided for each flower is : 
- sepal length
- sepal width
- petal length
- petal width

In [2]:
CSV_COLUMN_NAMES = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Versicolor','Virginica']


In [3]:
train_path = tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)

In [4]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [5]:
train_y = train.pop('Species') #We remove this column as this is our label we need to predict
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [6]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [7]:
train.shape

(120, 4)

---

## Input Function

**Reminder** :
For Linear regression the input function used was : 
```
def make_input_fn(data_df,label_df,num_epochs=50,shuffle=True,batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df)) # create tf.data.Dataset object with data and its label
        if shuffle:
            ds = ds.shuffle(1000) # randomize order of data
        ds = ds.batch(batch_size).repeat(num_epochs) # split dataset into batches of 32 and repeat process for number of epochs
        return ds # return a batch of the dataset
    return input_function # return a function object for use
```

In [8]:
def input_fn(features,labels,training=True,batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

---

## Feature Columns

In [9]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


---

## Building the Model

For classification tasks there are a variety of different estimators/models that we can pick from. Some of the famous options are : 
- ``` DNNClassifier ``` (Deep Neural Network)
- ``` LinearClassifier ```

We can chose either for the current model, but DNN is the better choice of the two as we may not be able to find a linear correspondance in our data

In [10]:
#Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    #two hidden layers of 30 and 10  nodes respectively.
    hidden_units=[30,10],
    #The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Tanay\\AppData\\Local\\Temp\\tmpmaa2yv4d', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


---

## Training

In [11]:
classifier.train(input_fn=lambda:input_fn(train,train_y,training=True),
                steps=6000)
#We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Tanay\AppData\Local\Temp\tmpmaa2yv4d\model.ckpt.
INFO:tensorflow:C:\Users\Tanay\AppData\Local\Temp\tmpmaa2yv4d\model.ckpt-0.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\Tanay\AppData\Local\Temp\tmpmaa2yv4d\model.ckpt-0.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\Tanay\AppData\Local\Temp\tmpmaa2yv4d\model.ckpt-0.meta
INFO:tensorflow:100
INF

In [12]:
eval_result = classifier.evaluate(
    input_fn=lambda:input_fn(test,test_y,training=False))

print('\nTest set accuracy : {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-08-03T20:52:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Tanay\AppData\Local\Temp\tmpmaa2yv4d\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.33087s
INFO:tensorflow:Finished evaluation at 2022-08-03-20:52:42
INFO:tensorflow:Saving dict for global step 6000: accuracy = 0.93333334, average_loss = 0.4433703, global_step = 6000, loss = 0.4433703
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6000: C:\Users\Tanay\AppData\Local\Temp\tmpmaa2yv4d\model.ckpt-6000

Test set accuracy : 0.933



---

## Predictions

In [13]:
def Input_fn(features,batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)


features = ['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
    valid = True
    while valid:
        val = input(feature+": ")
        if not val.isdigit(): valid = False
    
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda:Input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Predicition is "{}" ({:.1f}%)'.format(SPECIES[class_id],100*probability))

Please type numeric values as prompted.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Tanay\AppData\Local\Temp\tmpmaa2yv4d\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicition is "Virginica" (77.2%)
